In [1]:
# librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from pandas_profiling import ProfileReport
from IPython.display import HTML
from tqdm import tqdm

# ajustes para ver mejor todo:
pd.set_option('display.max_columns', None)
# pd.set_option('display.float_format', '{:.3f}'.format)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth',None)
# por si nos hace falta matplotlib
%matplotlib inline

In [2]:
df=pd.read_csv('./input/accidentalidad_completo_01.csv')
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)
df

c:\users\xhito\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado
0,2019S003289,01/01/2019,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0
1,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0
2,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0
3,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,2,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0
4,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 65 a 69 años,Mujer,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2020S014381,15/10/2020,Valle,CALL. GENERAL RICARDOS / CALL. NICOLAS MORALES,CARABANCHEL,Colisión fronto-lateral,Despejado,Autobús articulado EMT,Conductor,De 50 a 54 años,Hombre,"438105,1","4471335,37",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.913079,-3.774483,41.528200,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0
108691,2020S014381,15/10/2020,Valle,CALL. GENERAL RICARDOS / CALL. NICOLAS MORALES,CARABANCHEL,Colisión fronto-lateral,Despejado,Autobús articulado EMT,Pasajero,De 45 a 49 años,Hombre,"438105,1","4471335,37",N,1.0,0.0,2,0.0,0.0,1.0,0.0,-49.915812,-3.740653,27.454627,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0
108692,2020S014381,15/10/2020,Valle,CALL. GENERAL RICARDOS / CALL. NICOLAS MORALES,CARABANCHEL,Colisión fronto-lateral,Despejado,Autobús articulado EMT,Pasajero,Más de 74 años,Mujer,"438105,1","4471335,37",N,1.0,0.0,2,0.0,0.0,1.0,0.0,-49.874891,-3.797290,97.544663,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0
108693,2021S008475,16/05/2021,Valle,AVDA. LAS SUERTES / CALL. ENTREPEÑAS,VILLA DE VALLECAS,Alcance,Despejado,Moto de tres ruedas hasta 125cc,Conductor,De 30 a 34 años,Hombre,"448221,34","4469242,68",N,1.0,0.0,7,0.0,0.0,1.0,0.0,-49.825854,-3.809533,53.539622,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108695 entries, 0 to 108694
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   num_expediente        108695 non-null  object 
 1   fecha2                108695 non-null  object 
 2   hora                  108695 non-null  object 
 3   localizacion          108660 non-null  object 
 4   distrito              108690 non-null  object 
 5   tipo_accidente        108694 non-null  object 
 6   estado_meteorológico  97771 non-null   object 
 7   tipo_vehiculo         108289 non-null  object 
 8   tipo_persona          108685 non-null  object 
 9   rango_edad            108695 non-null  object 
 10  sexo                  108695 non-null  object 
 11  coordenada_x_utm      108695 non-null  object 
 12  coordenada_y_utm      108695 non-null  object 
 13  positiva_alcohol      108391 non-null  object 
 14  Interseccion          108695 non-null  float64
 15  

In [4]:
# Realizamos un groupby para saber la cantida de personas que se encuentran involucradas en cada accidente y remplazamos el nombre de la columna
n_victimas = df.groupby(["num_expediente"])["fecha2"].count().reset_index().rename(columns={"fecha2": "n_victimas"})

# Generamos un merge para añadir la columna n_victimas
df2 = pd.merge(df, n_victimas, left_on='num_expediente', right_on='num_expediente')
df2

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas
0,2019S003289,01/01/2019,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1
1,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4
2,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4
3,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,2,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4
4,2019S000022,01/01/2019,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 65 a 69 años,Mujer,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2019S036056,18/11/2019,Valle,CALL. GENERAL ARANAZ,CIUDAD LINEAL,Caída,Despejado,Patinete,Conductor,De 10 a 14 años,Hombre,"445920,59","4476956,02",N,0.0,0.0,3,0.0,0.0,1.0,0.0,-49.854133,-3.724303,35.480984,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,1
108691,2020S018936,21/12/2020,Valle,PASEO. DELICIAS,ARGANZUELA,Caída,Lluvia débil,Patinete,Conductor,De 25 a 29 años,Hombre,"441150,76","4473056,33",N,0.0,0.0,1,0.0,0.0,1.0,0.0,-49.876041,-3.789437,86.221415,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,2
108692,2020S018936,21/12/2020,Valle,PASEO. DELICIAS,ARGANZUELA,Caída,Lluvia débil,Patinete,Peatón,Desconocido,Desconocido,"441150,76","4473056,33",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.876041,-3.789437,86.221415,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,2
108693,2020S014102,09/10/2020,Valle,CALL. ISABEL PATACON / CALL. VICENTA JIMENEZ,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Maquinaria agrícola,Conductor,De 45 a 49 años,Hombre,"436500,67","4469148,53",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.954275,-3.849176,18.858436,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,1


In [5]:
# Transformamos la fecha a 'datetime64' para generar las medias a traves de los años y meses
df2['fecha2'] = pd.to_datetime(df2['fecha2'])
fecha = pd.to_datetime(df2['fecha2'])
dia_accidente = fecha.dt.day
mes_accidente = fecha.dt.month
año_accidente = fecha.dt.year
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108695 entries, 0 to 108694
Data columns (total 38 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   num_expediente        108695 non-null  object        
 1   fecha2                108695 non-null  datetime64[ns]
 2   hora                  108695 non-null  object        
 3   localizacion          108660 non-null  object        
 4   distrito              108690 non-null  object        
 5   tipo_accidente        108694 non-null  object        
 6   estado_meteorológico  97771 non-null   object        
 7   tipo_vehiculo         108289 non-null  object        
 8   tipo_persona          108685 non-null  object        
 9   rango_edad            108695 non-null  object        
 10  sexo                  108695 non-null  object        
 11  coordenada_x_utm      108695 non-null  object        
 12  coordenada_y_utm      108695 non-null  object        
 13 

In [6]:
# Creamos las columna mes y año
df2["dia"] = dia_accidente
df2['mes'] = mes_accidente
df2['año'] = año_accidente
df2.head(3)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019
1,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,4,1,1,2019
2,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,4,1,1,2019


In [7]:
# Obtenemos la media de precipitaciones por mes
media_precipitaciones = df2['prec'].groupby(df2['mes']).mean().reset_index()
media_precipitaciones.rename(columns={'prec':'media_mes_prec'},inplace=True)
df3 = pd.merge(df2, media_precipitaciones, on='mes')
df3

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984
1,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984
2,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984
3,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,2,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984
4,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 65 a 69 años,Mujer,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2019S039055,2019-12-20,Noche,GTA. JUAN DE HARO / AVDA. FRANCISCO JAVIER SAENZ DE OIZA,HORTALEZA,Vuelco,Lluvia débil,Cuadriciclo ligero,Conductor,De 18 a 20 años,Hombre,"446916,63","4483214,42",N,1.0,0.0,6,0.0,0.0,1.0,0.0,-49.877217,-3.808826,108.333336,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,1,20,12,2019,0.818918
108691,2020S019314,2020-12-28,Valle,AVDA. FELIPE II / CALL. NARVAEZ,SALAMANCA,Atropello a persona,Despejado,Sin especificar,Conductor,Desconocido,Desconocido,"442644,44","4475017,99",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.849200,-3.786408,54.090648,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,2,28,12,2020,0.818918
108692,2020S019314,2020-12-28,Valle,AVDA. FELIPE II / CALL. NARVAEZ,SALAMANCA,Atropello a persona,Despejado,Sin especificar,Peatón,De 25 a 29 años,Mujer,"442644,44","4475017,99",N,1.0,0.0,7,0.0,0.0,1.0,0.0,-49.849200,-3.786408,54.090648,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,2,28,12,2020,0.818918
108693,2020S018936,2020-12-21,Valle,PASEO. DELICIAS,ARGANZUELA,Caída,Lluvia débil,Patinete,Conductor,De 25 a 29 años,Hombre,"441150,76","4473056,33",N,0.0,0.0,1,0.0,0.0,1.0,0.0,-49.876041,-3.789437,86.221415,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,2,21,12,2020,0.818918


In [8]:
# IDEAS CUANTAS SON CALLES, AVDA, ETC...
# SEXO ENTRE NUM_EXPEDIENTE
# HORA ENTRE NUM_EXPEDIENTE


In [9]:
#Obtenemos el porcentaje de accidentes mensuales
media_accidentes = (df3.groupby(by = df3['mes']).count())['num_expediente'].reset_index()
media_accidentes['%accidente_mes']=(media_accidentes['num_expediente']/(media_accidentes['num_expediente'].sum()))*100
media_accidentes=media_accidentes[['mes','%accidente_mes']]
df4 = pd.merge(df3, media_accidentes, on='mes')
df4

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655
1,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655
2,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655
3,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,2,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655
4,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 65 a 69 años,Mujer,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2019S039055,2019-12-20,Noche,GTA. JUAN DE HARO / AVDA. FRANCISCO JAVIER SAENZ DE OIZA,HORTALEZA,Vuelco,Lluvia débil,Cuadriciclo ligero,Conductor,De 18 a 20 años,Hombre,"446916,63","4483214,42",N,1.0,0.0,6,0.0,0.0,1.0,0.0,-49.877217,-3.808826,108.333336,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,1,20,12,2019,0.818918,7.864207
108691,2020S019314,2020-12-28,Valle,AVDA. FELIPE II / CALL. NARVAEZ,SALAMANCA,Atropello a persona,Despejado,Sin especificar,Conductor,Desconocido,Desconocido,"442644,44","4475017,99",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.849200,-3.786408,54.090648,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,2,28,12,2020,0.818918,7.864207
108692,2020S019314,2020-12-28,Valle,AVDA. FELIPE II / CALL. NARVAEZ,SALAMANCA,Atropello a persona,Despejado,Sin especificar,Peatón,De 25 a 29 años,Mujer,"442644,44","4475017,99",N,1.0,0.0,7,0.0,0.0,1.0,0.0,-49.849200,-3.786408,54.090648,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,2,28,12,2020,0.818918,7.864207
108693,2020S018936,2020-12-21,Valle,PASEO. DELICIAS,ARGANZUELA,Caída,Lluvia débil,Patinete,Conductor,De 25 a 29 años,Hombre,"441150,76","4473056,33",N,0.0,0.0,1,0.0,0.0,1.0,0.0,-49.876041,-3.789437,86.221415,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,2,21,12,2020,0.818918,7.864207


In [10]:
edad_expediente=df4.groupby(df4['rango_edad'])['num_expediente'].count().reset_index()
edad_expediente['%edad']=(edad_expediente['num_expediente']/(edad_expediente['num_expediente'].sum()))*100
edad_expediente=edad_expediente[['rango_edad','%edad']]
df5=pd.merge(df4,edad_expediente,on='rango_edad')
df5

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%edad
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,10.798105
1,2019S000659,2019-01-01,Valle,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,De 25 a 29 años,Hombre,"442229,03","4475624,62",N,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-49.866272,-3.733635,38.182086,0.0,4.5,11.2,18.0,1.400000,7.200000,0,0,0,0,0,0,0,2,1,1,2019,1.184984,9.158655,10.798105
2,2019S000101,2019-01-01,Valle,CALL. FEDERICO GUTIERREZ / CALL. ELFO,CIUDAD LINEAL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,"445224,91","4476308,94",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.841938,-3.829920,70.128260,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,3,1,1,2019,1.184984,9.158655,10.798105
3,2019S000101,2019-01-01,Valle,CALL. FEDERICO GUTIERREZ / CALL. ELFO,CIUDAD LINEAL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,"445224,91","4476308,94",N,1.0,0.0,2,0.0,0.0,0.0,1.0,-49.841938,-3.829920,70.128260,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,3,1,1,2019,1.184984,9.158655,10.798105
4,2019S000101,2019-01-01,Valle,CALL. FEDERICO GUTIERREZ / CALL. ELFO,CIUDAD LINEAL,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 25 a 29 años,Hombre,"445224,91","4476308,94",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.863125,-3.762141,67.323056,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,3,1,1,2019,1.184984,9.158655,10.798105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2019S038586,2019-12-15,Valle,AVDA. POBLADOS / CALL. POLAN,CARABANCHEL,Colisión fronto-lateral,Despejado,Camión rígido,Pasajero,De 10 a 14 años,Hombre,"438291,15","4469475,19",N,1.0,0.0,2,0.0,0.0,1.0,0.0,-49.846280,-3.680626,14.048333,0.0,22.1,29.6,37.0,2.014238,9.147069,1,0,0,0,0,0,0,7,15,12,2019,0.818918,7.864207,0.969686
108691,2021S003759,2021-12-03,Punta,CALL. LUCHANA,CHAMBERÍ,Atropello a persona,Despejado,Motocicleta hasta 125cc,Peatón,De 10 a 14 años,Hombre,"440693,72","4475839,84",N,0.0,0.0,5,0.0,1.0,0.0,0.0,-49.902112,-3.780324,73.580497,0.0,20.0,26.0,32.0,3.300000,10.000000,1,0,0,0,0,0,0,2,3,12,2021,0.818918,7.864207,0.969686
108692,2019S039885,2019-12-22,Valle,CALL. GENERAL RICARDOS,CARABANCHEL,Caída,Despejado,Motocicleta hasta 125cc,Peatón,De 10 a 14 años,Hombre,"437738,42","4471024,65",N,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.899250,-3.711002,13.934852,0.0,24.5,32.2,40.0,1.900000,7.800000,1,0,0,0,0,0,0,2,22,12,2019,0.818918,7.864207,0.969686
108693,2021S014302,2021-12-08,Punta,CALL. SEPULVEDA,LATINA,Caída,Despejado,VMU eléctrico,Conductor,De 10 a 14 años,Mujer,"438108,48","4473132,14",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.900972,-3.823502,64.013034,1.6,9.2,10.6,12.0,1.400000,5.600000,0,0,0,0,1,0,0,2,8,12,2021,0.818918,7.864207,0.969686


In [11]:
accidentes_distrito=df4.groupby(df4['sexo'])['num_expediente'].count().reset_index()
accidentes_distrito['%sexo']=(accidentes_distrito['num_expediente']/(accidentes_distrito['num_expediente'].sum()))*100
accidentes_distrito=accidentes_distrito[['sexo','%sexo']]
df5=pd.merge(df4,accidentes_distrito,on='sexo')
df5

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284
1,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655,60.629284
2,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655,60.629284
3,2019S000022,2019-01-01,Valle,CALL. CONDE DE TORRALBA,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,"441944,8","4480964,51",N,0.0,0.0,2,0.0,0.0,0.0,1.0,-49.820947,-3.807079,44.627394,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655,60.629284
4,2019S000659,2019-01-01,Valle,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,SALAMANCA,Alcance,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,"442229,03","4475624,62",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.868998,-3.803924,112.686226,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,2,1,1,2019,1.184984,9.158655,60.629284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2019S036933,2019-12-11,Valle,PNTE. SEGOVIA / PASEO. EXTREMADURA,CENTRO,Atropello a persona,Lluvia débil,NaN,Conductor,Desconocido,Desconocido,"438678,19","4473955,72",N,1.0,0.0,14,NaN,NaN,NaN,NaN,-49.893032,-3.880136,30.071977,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,11,12,2019,0.818918,7.864207,10.643544
108691,2020S019008,2020-12-21,Valle,CALL. FRANCISCO RIOJA / CALL. LUIS RUIZ,CIUDAD LINEAL,Atropello a persona,Lluvia débil,NaN,Conductor,Desconocido,Desconocido,"445920,39","4475800,85",N,1.0,0.0,14,NaN,NaN,NaN,NaN,-49.891295,-3.797629,68.965619,0.0,18.4,26.1,33.8,2.200000,13.300000,1,0,0,0,0,0,0,2,21,12,2020,0.818918,7.864207,10.643544
108692,2020S019489,2020-12-31,Punta,CALL. ANTONIO DE LEYVA,CARABANCHEL,Alcance,Nublado,NaN,Conductor,Desconocido,Desconocido,"439232,59","4471235,4",NaN,0.0,0.0,14,NaN,NaN,NaN,NaN,-49.951375,-3.832939,25.118499,0.0,18.4,26.1,33.8,2.200000,13.300000,0,0,0,0,0,0,0,3,31,12,2020,0.818918,7.864207,10.643544
108693,2020S019314,2020-12-28,Valle,AVDA. FELIPE II / CALL. NARVAEZ,SALAMANCA,Atropello a persona,Despejado,Sin especificar,Conductor,Desconocido,Desconocido,"442644,44","4475017,99",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.849200,-3.786408,54.090648,0.0,2.3,6.8,11.4,0.300000,3.100000,1,0,0,0,0,0,0,2,28,12,2020,0.818918,7.864207,10.643544


In [12]:
# Descubrimos el rango de edad de las personas que han sufrido los accidentes
lista_edad=df5['rango_edad'].unique()
grupoe=[]
denoche=[]
depunta=[]
devalle=[]
for i in lista_edad:
    grupoe.append(i)
    edad=df5[df5['rango_edad']==i]
    noche=edad[edad['sexo']=='Hombre']
    denoche.append(noche['num_expediente'].count())
    punta=edad[edad['sexo']=='Mujer']
    depunta.append(punta['num_expediente'].count())
    valle=edad[edad['sexo']=='Desconocido']
    devalle.append(valle['num_expediente'].count())
    
df6=pd.DataFrame(list(zip(grupoe,denoche,depunta,devalle)),columns=['rango_edad','Hombre','Mujer','Desconocido'])
df6=df6.sort_values('rango_edad',ascending=True)
df6

,rango_edad,Hombre,Mujer,Desconocido
15,De 10 a 14 años,549,505,0
11,De 15 a 17 años,442,367,0
7,De 18 a 20 años,2123,1182,1
10,De 21 a 24 años,4905,2400,0
0,De 25 a 29 años,7702,4034,1
8,De 30 a 34 años,7638,3740,3
4,De 35 a 39 años,7707,3511,0
1,De 40 a 44 años,8283,3400,0
2,De 45 a 49 años,7256,3145,0
5,De 50 a 54 años,6257,2570,1


In [13]:
df7=df5.groupby(df5['rango_edad'])['num_expediente'].count().reset_index()
df7=df7.sort_values('rango_edad',ascending=True)
df7

,rango_edad,num_expediente
0,De 10 a 14 años,1054
1,De 15 a 17 años,809
2,De 18 a 20 años,3306
3,De 21 a 24 años,7305
4,De 25 a 29 años,11737
5,De 30 a 34 años,11381
6,De 35 a 39 años,11218
7,De 40 a 44 años,11683
8,De 45 a 49 años,10401
9,De 50 a 54 años,8828


In [14]:
#aparicion de la virgen es un concat de libro...pero no funciona
df8=pd.merge(df6,df7,on='rango_edad')
df8

,rango_edad,Hombre,Mujer,Desconocido,num_expediente
0,De 10 a 14 años,549,505,0,1054
1,De 15 a 17 años,442,367,0,809
2,De 18 a 20 años,2123,1182,1,3306
3,De 21 a 24 años,4905,2400,0,7305
4,De 25 a 29 años,7702,4034,1,11737
5,De 30 a 34 años,7638,3740,3,11381
6,De 35 a 39 años,7707,3511,0,11218
7,De 40 a 44 años,8283,3400,0,11683
8,De 45 a 49 años,7256,3145,0,10401
9,De 50 a 54 años,6257,2570,1,8828


In [15]:
# Creamos los % de las columnas 
df8['%hombre']=df8['Hombre']/(df8['num_expediente'])*100
df8['%Mujer']=df8['Mujer']/(df8['num_expediente'])*100
df8.drop(['Desconocido','num_expediente'],axis=1,inplace=True)
df8

,rango_edad,Hombre,Mujer,%hombre,%Mujer
0,De 10 a 14 años,549,505,52.087287,47.912713
1,De 15 a 17 años,442,367,54.635352,45.364648
2,De 18 a 20 años,2123,1182,64.216576,35.753176
3,De 21 a 24 años,4905,2400,67.145791,32.854209
4,De 25 a 29 años,7702,4034,65.621539,34.369941
5,De 30 a 34 años,7638,3740,67.111853,32.861787
6,De 35 a 39 años,7707,3511,68.702086,31.297914
7,De 40 a 44 años,8283,3400,70.897886,29.102114
8,De 45 a 49 años,7256,3145,69.762523,30.237477
9,De 50 a 54 años,6257,2570,70.876756,29.111917


In [16]:
df9=pd.merge(df5,df8,on='rango_edad')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941
1,2019S000659,2019-01-01,Valle,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,De 25 a 29 años,Hombre,"442229,03","4475624,62",N,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-49.866272,-3.733635,38.182086,0.0,4.5,11.2,18.0,1.4,7.2,0,0,0,0,0,0,0,2,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941
2,2019S000101,2019-01-01,Valle,CALL. FEDERICO GUTIERREZ / CALL. ELFO,CIUDAD LINEAL,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 25 a 29 años,Hombre,"445224,91","4476308,94",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.863125,-3.762141,67.323056,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,3,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941
3,2019S000009,2019-01-01,Punta,CALL. REAL DE ARGANDA / AVDA. LA GAVIA,VILLA DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 25 a 29 años,Hombre,"447254,54","4470102,32",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.893688,-3.769355,43.776974,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,4,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941
4,2020S000060,2020-01-01,Valle,GTA. MARIANO SALVADOR MAELLA,FUENCARRAL-EL PARDO,Caída,Despejado,Motocicleta hasta 125cc,Conductor,De 25 a 29 años,Hombre,"438797,72","4480843,05",N,0.0,0.0,7,0.0,1.0,0.0,0.0,-49.946836,-3.820125,30.527380,0.0,3.7,8.4,13.0,3.9,15.3,0,0,0,0,0,0,0,2,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108690,2020S019184,2020-12-25,Punta,AUTOV. A-3,PUENTE DE VALLECAS,Solo salida de la vía,Despejado,Turismo,Pasajero,De 6 a 9 años,Mujer,"444950,58","4472305,63",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.889348,-3.813730,79.063823,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,4,25,12,2020,0.818918,7.864207,28.727172,365,331,52.367288,47.489240
108691,2020S019233,2020-12-26,Valle,CALL. DIAMANTE / CALL. SELMA LAGERLOFF,VILLAVERDE,Colisión lateral,Despejado,Turismo,Pasajero,De 6 a 9 años,Mujer,"441522,69","4467287,99",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.911027,-3.901520,31.634082,5.9,-0.5,2.0,4.4,0.6,5.3,0,0,0,0,1,0,0,4,26,12,2020,0.818918,7.864207,28.727172,365,331,52.367288,47.489240
108692,2019S039734,2019-12-29,Valle,CTRA. VILLAVERDE A VALLECAS,VILLAVERDE,Alcance,Despejado,Turismo,Pasajero,De 6 a 9 años,Mujer,"441521,63","4467343,52",N,0.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.910308,-3.758034,44.953185,0.0,1.4,5.4,9.4,3.6,10.6,0,0,0,0,0,0,0,5,29,12,2019,0.818918,7.864207,28.727172,365,331,52.367288,47.489240
108693,2019S038586,2019-12-15,Valle,AVDA. POBLADOS / CALL. POLAN,CARABANCHEL,Colisión fronto-lateral,Despejado,Autobús,Pasajero,De 6 a 9 años,Mujer,"438291,15","4469475,19",N,1.0,0.0,7,0.0,0.0,1.0,0.0,-49.872178,-3.815440,123.198913,0.0,21.6,29.0,36.4,2.5,6.7,1,0,0,0,0,0,0,7,15,12,2019,0.818918,7.864207,28.727172,365,331,52.367288,47.489240


In [17]:
datosCC = df9.groupby("distrito")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosCC

,distrito,sum,mean,std,amax,amin
0,MORATALAZ,174372.337837,55.960314,32.505483,126.350828,0.281105
1,CIUDAD LINEAL,399782.091622,55.804312,32.214192,126.350828,0.103044
2,CHAMBERÍ,297907.208442,55.683590,31.953297,126.227475,0.055007
3,HORTALEZA,250006.573227,55.421541,32.024577,126.302968,0.153836
4,BARAJAS,110615.124934,55.390648,31.334313,125.414458,0.386575
5,CARABANCHEL,398335.144336,55.308962,32.116892,126.307674,0.261048
6,VICÁLVARO,89994.895190,55.245485,32.062135,126.227475,0.497908
7,TETUÁN,274994.112657,55.120087,32.050729,126.307674,0.189648
8,SAN BLAS-CANILLEJAS,319219.739905,55.028399,31.792648,126.350828,0.435341
9,PUENTE DE VALLECAS,477328.088928,54.991715,31.974564,126.350828,0.128234


In [18]:
df9=pd.merge(df9, datosCC, on='distrito')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,sum,mean,std,amax,amin
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048
1,2020S000006,2020-01-01,Noche,CALL. EUGENIA DE MONTIJO,CARABANCHEL,Colisión lateral,Despejado,Motocicleta hasta 125cc,Conductor,De 25 a 29 años,Hombre,"436671,67","4470229,71",S,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.946836,-3.820125,30.527380,0.0,3.7,8.4,13.0,3.9,15.3,0,0,0,0,0,0,0,4,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048
2,2020S000020,2020-01-01,Punta,CALL. PARADOR DEL SOL / CALL. FERNAN CABALLERO,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439683,32","4471490,46",N,1.0,0.0,7,0.0,0.0,0.0,1.0,-49.821726,-3.850844,28.724542,0.0,1.9,6.6,11.2,0.3,3.3,1,0,0,0,0,0,0,3,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048
3,2020S000020,2020-01-01,Punta,CALL. PARADOR DEL SOL / CALL. FERNAN CABALLERO,CARABANCHEL,Colisión fronto-lateral,Despejado,Todo terreno,Conductor,De 25 a 29 años,Hombre,"439683,32","4471490,46",N,1.0,0.0,14.0,0.0,0.0,1.0,0.0,-49.938102,-3.783374,6.167147,9.6,11.3,13.7,16.1,1.9,8.9,0,1,0,0,0,0,0,3,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048
4,2021S005270,2021-01-04,Noche,CALL. GAVIOTA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437438,61","4471146,83",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.914770,-3.830673,52.582825,2.2,8.3,11.8,15.2,4.2,13.9,0,0,0,0,0,0,0,2,4,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108685,2021S008614,2021-05-19,Punta,AUTOV. M-23,MORATALAZ,Colisión múltiple,Despejado,Turismo,Pasajero,De 6 a 9 años,Mujer,"444769,9","4474188,44",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.900132,-3.790298,118.681935,0.0,15.7,22.5,29.3,2.2,11.4,1,0,0,0,0,0,0,4,19,5,2021,0.868112,9.293896,28.727172,365,331,52.367288,47.489240,174372.337837,55.960314,32.505483,126.350828,0.281105
108686,2019S034881,2019-06-11,Punta,M-23,MORATALAZ,Alcance,Despejado,Turismo,Pasajero,De 6 a 9 años,Mujer,"444769,9","4474188,44",N,0.0,1.0,14,0.0,0.0,0.0,1.0,-49.923648,-3.735494,22.738351,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,8,11,6,2019,0.945450,9.660058,28.727172,365,331,52.367288,47.489240,174372.337837,55.960314,32.505483,126.350828,0.281105
108687,2021S010925,2021-06-17,Valle,CALL. CAÑON DEL RIO LOBOS / CALL. LAGUNA NEGRA,MORATALAZ,Colisión múltiple,Lluvia débil,Turismo,Pasajero,De 6 a 9 años,Mujer,"446349,23","4472898,54",N,1.0,0.0,7,0.0,0.0,0.0,1.0,-49.894754,-3.843216,56.667307,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,6,17,6,2021,0.945450,9.660058,28.727172,365,331,52.367288,47.489240,174372.337837,55.960314,32.505483,126.350828,0.281105
108688,2019S020591,2019-07-05,Valle,AUTOV. M-23,MORATALAZ,Colisión múltiple,Despejado,Tur

In [19]:
df9["distrito_desnsidad_ponderado"]= df9["densidad_accidente"] / (df9["amax"] - df9["amin"]) * 100
df9.sort_values(by="distrito_desnsidad_ponderado",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,sum,mean,std,amax,amin,distrito_desnsidad_ponderado
95487,2020S011037,2020-07-08,Punta,CALL. HERCE / CALL. GALLO,VICÁLVARO,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,"447953,03","4473040,53",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.872933,-3.811643,126.227475,0.0,15.5,21.6,27.7,3.3,15.6,1,0,0,0,0,0,0,3,8,7,2020,1.425777,9.017894,10.643544,310,156,2.577748,1.297189,89994.895190,55.245485,32.062135,126.227475,0.497908,100.396015
40202,2019S034346,2019-10-30,Punta,PASEO. EXTREMADURA / CALL. LOS YEBENES,LATINA,Alcance,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,"435625,91","4472695,39",N,1.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.872933,-3.811643,126.227475,16.2,17.7,24.0,30.2,0.8,13.6,0,0,0,0,0,0,0,2,30,10,2019,1.182292,7.873407,60.629284,3072,1246,71.144048,28.855952,289168.113428,54.694177,31.788146,126.227475,0.458738,100.364747
39808,2019S018414,2019-09-04,Valle,AVDA. PORTUGAL,LATINA,Alcance,Nublado,Furgoneta,Conductor,De 30 a 34 años,Hombre,"437680,56","4473793,05",N,0.0,0.0,5,1.0,0.0,0.0,0.0,-49.872401,-3.812173,126.222732,28.5,5.4,8.4,11.4,3.1,14.4,0,0,0,0,0,0,0,3,4,9,2019,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,289168.113428,54.694177,31.788146,126.227475,0.458738,100.360976
41024,2021S008454,2021-05-13,Punta,CALL. SAN JUAN DE MATA,LATINA,Colisión lateral,NaN,Todo terreno,Conductor,Desconocido,Desconocido,"435765,9","4472975,53",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.872401,-3.812173,126.222732,0.0,5.2,9.2,13.1,4.4,17.5,1,0,0,0,0,0,0,2,13,5,2021,0.868112,9.293896,10.643544,310,156,2.577748,1.297189,289168.113428,54.694177,31.788146,126.227475,0.458738,100.360976
36671,2019S020209,2019-04-27,Noche,CALL. SESEÑA,LATINA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"435366,86","4472253,84",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.872401,-3.812173,126.222732,0.0,6.3,11.2,16.0,2.8,10.8,0,0,0,0,0,0,0,3,27,4,2019,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,289168.113428,54.694177,31.788146,126.227475,0.458738,100.360976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77178,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,"436511,35","4478908,8",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.5,12.8,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,60.629284,5037,1928,72.318737,27.681263,309700.856198,54.862862,32.242187,126.307674,0.082441,0.065312
33772,2019S025634,2019-04-07,Punta,CALL. DOCTOR ESQUERDO,RETIRO,Colisión múltiple,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,"443257,61","4474572,47",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.919575,-3.964384,0.060811,4.5,16.0,22.0,28.0,2.8,11.7,0,0,0,0,0,0,0,4,7,4,2019,1.345597,7.649846,28.727172,7707,3511,68.702086,31.297914,294465.469939,54.169512,32.126548,126.350828,0.041224,0.048145
69321,2020S006970,2020-05-18,Punta,PLAZA. CRISTO REY / AVDA. REYES CATOLICOS,CHAMBERÍ,Colisión fronto-lateral,Despejado,Motocicleta > 125cc,Conductor,De 60 a 64 años,Hombre,"439179,79","4476706,36",N,1.0,0.0,14,0.0,1.0,0.0,0.0,-49.784818,-3.890852,0.055007,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,3,18,5,2020,0.868112,9.293896,60.629284,3072,1246,71.144048,28.855952,297907.208442,55.683590,31.953297,126.227475,0.055007,0.043597
8435,2019S037

In [20]:
datosTV = df9.groupby("tipo_vehiculo")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosTV

,tipo_vehiculo,sum,mean,std,amax,amin
0,Ambulancia SAMUR,8.089086e+02,89.878739,34.937087,124.921428,19.344149
1,Semiremolque,5.208138e+02,65.101724,33.493844,103.217469,26.429901
2,Cuadriciclo no ligero,2.182994e+03,64.205717,25.867715,95.179412,12.599751
3,Autobús articulado,1.032257e+04,58.319622,31.840899,121.557932,3.510344
4,Patinete,6.941293e+02,57.844109,27.742912,93.761601,35.480984
5,Vehículo articulado,1.202395e+04,57.530885,27.498849,123.157631,2.703069
6,Tractocamión,2.643575e+04,56.607596,31.232733,120.169125,4.209194
7,Maquinaria de obras,1.497816e+04,56.097965,30.403105,126.039514,4.700452
8,VMU eléctrico,3.228313e+04,55.756695,32.176555,124.640895,0.105714
9,Bicicleta EPAC (pedaleo asistido),1.268010e+04,55.614458,33.637916,123.157631,1.600969


In [21]:
df9=pd.merge(df9, datosTV, on='tipo_vehiculo')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,sum_x,mean_x,std_x,amax_x,amin_x,distrito_desnsidad_ponderado,sum_y,mean_y,std_y,amax_y,amin_y
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048,21.926341,4.138136e+06,54.922498,31.948221,126.350828,0.041224
1,2020S000020,2020-01-01,Punta,CALL. PARADOR DEL SOL / CALL. FERNAN CABALLERO,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439683,32","4471490,46",N,1.0,0.0,7,0.0,0.0,0.0,1.0,-49.821726,-3.850844,28.724542,0.0,1.9,6.6,11.2,0.300000,3.300000,1,0,0,0,0,0,0,3,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048,22.788823,4.138136e+06,54.922498,31.948221,126.350828,0.041224
2,2021S005270,2021-01-04,Noche,CALL. GAVIOTA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437438,61","4471146,83",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.914770,-3.830673,52.582825,2.2,8.3,11.8,15.2,4.200000,13.900000,0,0,0,0,0,0,0,2,4,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048,41.716963,4.138136e+06,54.922498,31.948221,126.350828,0.041224
3,2021S009612,2021-01-06,Punta,CALL. ANTONIO LOPEZ,CARABANCHEL,Colisión múltiple,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439400,11","4472079,69",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.838265,-3.824065,77.416569,0.0,13.5,19.8,26.0,1.900000,10.800000,1,0,0,0,0,0,0,3,6,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048,61.418993,4.138136e+06,54.922498,31.948221,126.350828,0.041224
4,2021S009612,2021-01-06,Punta,CALL. ANTONIO LOPEZ,CARABANCHEL,Colisión múltiple,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439400,11","4472079,69",N,0.0,0.0,1,0.0,0.0,0.0,1.0,-49.838265,-3.824065,77.416569,0.0,13.5,19.8,26.0,1.900000,10.800000,1,0,0,0,0,0,0,3,6,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,398335.144336,55.308962,32.116892,126.307674,0.261048,61.418993,4.138136e+06,54.922498,31.948221,126.350828,0.041224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108279,2020S014347,2020-10-13,Valle,CALL. FEDERICO SALMON,CHAMARTÍN,Alcance,NaN,Ambulancia SAMUR,Pasajero,De 40 a 44 años,Hombre,"443391,04","4479374,64",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.908653,-3.793580,93.707820,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,4,13,10,2020,1.182292,7.873407,60.629284,8283,3400,70.897886,29.102114,411327.409975,54.705068,31.732026,126.350828,0.194485,74.279119,8.089086e+02,89.878739,34.937087,124.921428,19.344149
108280,2020S014347,2020-10-13,Valle,CALL. FEDERICO SALMON,CHAMARTÍN,Alcance,NaN,Ambulancia SAMUR,Pasajero,De 70 a 74 años,Mujer,"443391,04","4479374,64",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.874858,-3.812754,124.921428,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,4,13,10,2020,1.182292,7.873407,28.727172,938,565,62.367021,37.566489,411327.409975,54.705068,31.7

In [22]:
df9["tipo_vehiculo_densidad_ponderado"]= df9["densidad_accidente"] / (df9["amax_y"] - df9["amin_y"]) * 100
df9.sort_values(by="tipo_vehiculo_densidad_ponderado",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,sum_x,mean_x,std_x,amax_x,amin_x,distrito_desnsidad_ponderado,sum_y,mean_y,std_y,amax_y,amin_y,tipo_vehiculo_densidad_ponderado
108283,2019S018939,2019-04-15,Valle,CALL. ARQUITECTO SANCHEZ ARCAS,MONCLOA-ARAVACA,Alcance,Despejado,Tranvía,Conductor,Desconocido,Desconocido,"439306,52","4478383,07",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.913079,-3.774483,41.528200,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,2,15,4,2019,1.345597,7.649846,10.643544,310,156,2.577748,1.297189,309700.856198,54.862862,32.242187,126.307674,0.082441,32.900078,4.152820e+01,41.528200,NaN,41.528200,41.528200,inf
108228,2020S011527,2020-08-22,Valle,CALL. ALCALA,CIUDAD LINEAL,Caída,Despejado,Moto de tres ruedas hasta 125cc,Conductor,De 25 a 29 años,Mujer,"445207,35","4476127,24",N,0.0,0.0,5,0.0,0.0,1.0,0.0,-49.825854,-3.809533,53.539622,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,1,22,8,2020,1.223388,7.157643,28.727172,7702,4034,65.621539,34.369941,399782.091622,55.804312,32.214192,126.350828,0.103044,42.408366,1.070792e+02,53.539622,0.000000,53.539622,53.539622,inf
108229,2021S008475,2021-05-16,Valle,AVDA. LAS SUERTES / CALL. ENTREPEÑAS,VILLA DE VALLECAS,Alcance,Despejado,Moto de tres ruedas hasta 125cc,Conductor,De 30 a 34 años,Hombre,"448221,34","4469242,68",N,1.0,0.0,7,0.0,0.0,1.0,0.0,-49.825854,-3.809533,53.539622,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,4,16,5,2021,0.868112,9.293896,60.629284,7638,3740,67.111853,32.861787,157988.790790,54.535309,32.594893,126.227475,0.153836,42.466944,1.070792e+02,53.539622,0.000000,53.539622,53.539622,inf
108057,2020S015160,2020-10-27,Valle,AUTOV. M-30,PUENTE DE VALLECAS,Alcance,Despejado,Maquinaria agrícola,Conductor,De 40 a 44 años,Hombre,"442175,07","4470727,89",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.835665,-3.767225,26.429901,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,3,27,10,2020,1.182292,7.873407,60.629284,8283,3400,70.897886,29.102114,477328.088928,54.991715,31.974564,126.350828,0.128234,20.939121,8.300521e+01,20.751303,3.785732,26.429901,18.858436,349.072495
108055,2020S014102,2020-09-10,Valle,CALL. ISABEL PATACON / CALL. VICENTA JIMENEZ,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Maquinaria agrícola,Conductor,De 45 a 49 años,Hombre,"436500,67","4469148,53",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.954275,-3.849176,18.858436,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,1,1,10,9,2020,1.937179,7.047242,60.629284,7256,3145,69.762523,30.237477,398335.144336,55.308962,32.116892,126.307674,0.261048,14.961476,8.300521e+01,20.751303,3.785732,26.429901,18.858436,249.072495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,"436511,35","4478908,8",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.500000,12.800000,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,60.629284,5037,1928,72.318737,27.681263,309700.856198,54.862862,32.242187,126.307674,0.082441,0.065312,4.138136e+06,54.922498,31.948221,126.350828,0.041224,0.065269
23529,2019S025634,2019-04-07,Punta,CALL. DOCTOR ESQUERDO,RETIRO,Colisión múltiple,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,"443257,61","4474572,47",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.919575,-3.964384,0.060811,4.5,16.0,

In [23]:
# eliminamos columnas que han surgido
df9.drop(['sum_x','mean_x','std_x','amax_x','amin_x','sum_y','mean_y','std_y','amax_y','amin_y'],axis=1,inplace=True)
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691
1,2020S000020,2020-01-01,Punta,CALL. PARADOR DEL SOL / CALL. FERNAN CABALLERO,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439683,32","4471490,46",N,1.0,0.0,7,0.0,0.0,0.0,1.0,-49.821726,-3.850844,28.724542,0.0,1.9,6.6,11.2,0.300000,3.300000,1,0,0,0,0,0,0,3,1,1,2020,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,22.788823,22.741376
2,2021S005270,2021-01-04,Noche,CALL. GAVIOTA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437438,61","4471146,83",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.914770,-3.830673,52.582825,2.2,8.3,11.8,15.2,4.200000,13.900000,0,0,0,0,0,0,0,2,4,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,41.716963,41.630108
3,2021S009612,2021-01-06,Punta,CALL. ANTONIO LOPEZ,CARABANCHEL,Colisión múltiple,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439400,11","4472079,69",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.838265,-3.824065,77.416569,0.0,13.5,19.8,26.0,1.900000,10.800000,1,0,0,0,0,0,0,3,6,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,61.418993,61.291118
4,2021S009612,2021-01-06,Punta,CALL. ANTONIO LOPEZ,CARABANCHEL,Colisión múltiple,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"439400,11","4472079,69",N,0.0,0.0,1,0.0,0.0,0.0,1.0,-49.838265,-3.824065,77.416569,0.0,13.5,19.8,26.0,1.900000,10.800000,1,0,0,0,0,0,0,3,6,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,61.418993,61.291118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108279,2020S014347,2020-10-13,Valle,CALL. FEDERICO SALMON,CHAMARTÍN,Alcance,NaN,Ambulancia SAMUR,Pasajero,De 40 a 44 años,Hombre,"443391,04","4479374,64",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.908653,-3.793580,93.707820,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,4,13,10,2020,1.182292,7.873407,60.629284,8283,3400,70.897886,29.102114,74.279119,88.757563
108280,2020S014347,2020-10-13,Valle,CALL. FEDERICO SALMON,CHAMARTÍN,Alcance,NaN,Ambulancia SAMUR,Pasajero,De 70 a 74 años,Mujer,"443391,04","4479374,64",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.874858,-3.812754,124.921428,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,4,13,10,2020,1.182292,7.873407,28.727172,938,565,62.367021,37.566489,99.021124,118.322265
108281,2021S006109,2021-04-13,Noche,CALL. SAN BERNARDO,CENTRO,Alcance,Despejado,Ambulancia SAMUR,Conductor,De 30 a 34 años,Hombre,"439999,37","4475137,76",N,0.0,0.0,7,0.0,0.0,1.0,0.0,-49.908653,-3.793580,93.707820,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,2,13,4,2021,1.345597,7.649846,60.629284,7638,3740,67.111853,32.861787,74.381988,88.757563
108282,2021S006335,2021-04-17,Valle,AVDA. DEMOCRACIA,VICÁLVARO,Alcance,Despejado,Ambulancia SAMUR,Conductor,Desconocido,Desconocido,"447231,75","4471429,83",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.874858,-3.812754,124.921428,1.0,-1.2,2.8,6.9,2.014238,9.147069,0,0,0,0,0,0,0,2,17,4,2021,1.345597,7.649846,10.643544

In [24]:
datosTA = df9.groupby("tipo_accidente")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosTA

,tipo_accidente,sum,mean,std,amax,amin
0,Despeñamiento,6.390268e+02,63.902684,23.098671,96.730408,13.901156
1,Atropello a animal,1.072344e+04,58.279556,31.515483,124.650603,4.209194
2,Solo salida de la vía,2.177620e+04,57.305792,31.033757,120.935777,2.508042
3,Colisión múltiple,4.280375e+05,55.488393,32.190606,126.265660,0.060811
4,Colisión frontal,1.409664e+05,55.454927,31.430913,126.302968,0.194485
5,Colisión fronto-lateral,1.474194e+06,55.116244,31.791005,126.350828,0.055007
6,Caída,3.729869e+05,55.085934,32.201867,126.307674,0.041220
7,Choque contra obstáculo fijo,7.595644e+05,54.862001,32.069455,126.307674,0.103044
8,Colisión lateral,8.337262e+05,54.861235,31.785113,126.350828,0.041224
9,Atropello a persona,3.900391e+05,54.696272,31.893957,126.208812,0.261048


In [25]:
df9=pd.merge(df9, datosTA, on='tipo_accidente')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,sum,mean,std,amax,amin
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,10723.438379,58.279556,31.515483,124.650603,4.209194
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,10723.438379,58.279556,31.515483,124.650603,4.209194
2,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,10723.438379,58.279556,31.515483,124.650603,4.209194
3,2020S010049,2020-07-18,Valle,CALL. ARROYO OPAÑEL,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,"439025,37","4471090,28",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.864850,-3.846994,71.213893,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,18,7,2020,1.425777,9.017894,28.727172,7702,4034,65.621539,34.369941,56.498056,56.380427,10723.438379,58.279556,31.515483,124.650603,4.209194
4,2019S039887,2019-12-23,Valle,PASEO. ERMITA DEL SANTO,CARABANCHEL,Atropello a animal,NaN,Turismo,Conductor,De 40 a 44 años,Hombre,"438420,26","4472476,13",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.909412,-3.843681,66.684388,0.6,7.8,13.1,18.4,1.900000,7.500000,0,0,0,0,0,0,0,1,23,12,2019,0.818918,7.864207,60.629284,8283,3400,70.897886,29.102114,52.904540,52.794393,10723.438379,58.279556,31.515483,124.650603,4.209194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108278,2019S032345,2019-09-20,Valle,PLAZA. JUAN ANTONIO SUANZES,SAN BLAS-CANILLEJAS,Despeñamiento,Despejado,Turismo,Conductor,Desconocido,Desconocido,"447058,48","4477358,09",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.851610,-3.807776,82.862030,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,20,9,2019,1.937179,7.047242,10.643544,310,156,2.577748,1.297189,65.807656,65.602320,639.026837,63.902684,23.098671,96.730408,13.901156
108279,2019S032345,2019-09-20,Valle,PLAZA. JUAN ANTONIO SUANZES,SAN BLAS-CANILLEJAS,Despeñamiento,Despejado,Turismo,Conductor,Desconocido,Desconocido,"447058,48","4477358,09",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.851610,-3.807776,82.862030,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,20,9,2019,1.937179,7.047242,10.643544,310,156,2.577748,1.297189,65.807656,65.602320,639.026837,63.902684,23.098671,96.730408,13.901156
108280,2019S031644,2019-09-30,Valle,CALL. CINCA,CHAMARTÍN,Despeñamiento,Despejado,Camión rígido,Conductor,De 25 a 29 años,Hombre,"442360,08","4477858,05",N,0.0,0.0,14,0.0,0.0,1.0,0.0,-49.887949,-3.813770,79.129520

In [26]:
df9["tipo_accidente_densidad_ponderado"]= df9["densidad_accidente"] / (df9["amax"] - df9["amin"]) * 100
df9.sort_values(by="tipo_accidente_densidad_ponderado",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,sum,mean,std,amax,amin,tipo_accidente_densidad_ponderado
108276,2019S031644,2019-09-30,Valle,CALL. CINCA,CHAMARTÍN,Despeñamiento,Despejado,Turismo,Conductor,Desconocido,Desconocido,"442360,08","4477858,05",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.858214,-3.800525,96.730408,6.2,23.8,30.9,38.0,1.9,13.3,0,1,0,0,0,0,0,2,30,9,2019,1.937179,7.047242,10.643544,310,156,2.577748,1.297189,76.675026,76.581991,6.390268e+02,63.902684,23.098671,96.730408,13.901156,116.782906
35,2021S014875,2021-08-26,Punta,CTRA. M-612,FUENCARRAL-EL PARDO,Atropello a animal,NaN,Turismo,Conductor,De 45 a 49 años,Hombre,"435871,19","4484913,01",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.871832,-3.814140,124.650603,0.1,14.0,16.9,19.8,2.5,9.7,0,0,0,0,0,0,1,1,26,8,2021,1.223388,7.157643,60.629284,7256,3145,69.762523,30.237477,98.785115,98.686560,1.072344e+04,58.279556,31.515483,124.650603,4.209194,103.494806
108213,2019S034691,2019-03-11,Valle,CALL. VELAZQUEZ / CALL. MARIA DE MOLINA,SALAMANCA,Solo salida de la vía,Despejado,Motocicleta hasta 125cc,Conductor,De 25 a 29 años,Hombre,"442078,82","4476523,61",N,1.0,0.0,6,0.0,1.0,0.0,0.0,-49.876364,-3.813067,120.935777,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,11,3,2019,1.564984,7.984728,60.629284,7702,4034,65.621539,34.369941,95.794195,95.965933,2.177620e+04,57.305792,31.033757,120.935777,2.508042,102.117783
33,2019S024325,2019-06-16,Valle,CALL. SAN DACIO,FUENCARRAL-EL PARDO,Atropello a animal,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,"441662,17","4481879,36",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.875726,-3.812936,122.938605,26.6,15.0,19.7,24.4,1.4,9.7,0,0,0,0,1,0,0,4,16,6,2019,0.945450,9.660058,60.629284,7256,3145,69.762523,30.237477,97.428363,97.331162,1.072344e+04,58.279556,31.515483,124.650603,4.209194,102.073369
107905,2020S015649,2020-04-11,Valle,AVDA. AMERICA / AUTOV. M-30,SALAMANCA,Solo salida de la vía,Lluvia débil,Turismo,Conductor,De 45 a 49 años,Hombre,"443834,32","4477155,4",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.901856,-3.791500,120.486281,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,11,4,2020,1.345597,7.649846,60.629284,7256,3145,69.762523,30.237477,95.438147,95.389644,2.177620e+04,57.305792,31.033757,120.935777,2.508042,101.738231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86807,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,"436511,35","4478908,8",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.5,12.8,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,60.629284,5037,1928,72.318737,27.681263,0.065312,0.065269,1.376888e+06,54.657915,32.067828,126.350828,0.082441,0.065290
42896,2019S025634,2019-04-07,Punta,CALL. DOCTOR ESQUERDO,RETIRO,Colisión múltiple,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,"443257,61","4474572,47",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.919575,-3.964384,0.060811,4.5,16.0,22.0,28.0,2.8,11.7,0,0,0,0,0,0,0,4,7,4,2019,1.345597,7.649846,28.727172,7707,3511,68.702086,31.297914,0.048145,0.048145,4.280375e+05,55.488393,32.190606,126.265660,0.060811,0.048185
24057,2020S006970,2020-05-18,Punta,PLAZA. CRISTO REY / AVDA. REYES CATOLICOS,CHAMBERÍ,Colisión fronto-lateral,Despejado,Motocicleta > 125cc,Conductor,De 60 a 64 años,Hombre,"439179,79","4476706,36

In [27]:
datosHR = df9.groupby("hora")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosHR

,hora,sum,mean,std,amax,amin
0,Punta,1.970405e+06,55.048458,31.883662,126.350828,0.041220
1,Valle,3.265187e+06,54.857724,31.963052,126.350828,0.082441
2,Noche,7.104302e+05,54.783331,31.938207,126.350828,0.105714


In [28]:
df9=pd.merge(df9, datosHR, on='hora')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,sum_x,mean_x,std_x,amax_x,amin_x,tipo_accidente_densidad_ponderado,sum_y,mean_y,std_y,amax_y,amin_y
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,10723.438379,58.279556,31.515483,124.650603,4.209194,22.946770,7.104302e+05,54.783331,31.938207,126.350828,0.105714
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,10723.438379,58.279556,31.515483,124.650603,4.209194,44.545513,7.104302e+05,54.783331,31.938207,126.350828,0.105714
2,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,10723.438379,58.279556,31.515483,124.650603,4.209194,8.018739,7.104302e+05,54.783331,31.938207,126.350828,0.105714
3,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Peatón,De 35 a 39 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,7707,3511,68.702086,31.297914,46.215468,46.119247,10723.438379,58.279556,31.515483,124.650603,4.209194,48.366287,7.104302e+05,54.783331,31.938207,126.350828,0.105714
4,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,10723.438379,58.279556,31.515483,124.650603,4.209194,48.366287,7.104302e+05,54.783331,31.938207,126.350828,0.105714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108278,2019S032026,2019-04-10,Punta,AUTOV. A-3,PUENTE DE VALLECAS,Solo salida de la vía,Despejado,Motocicleta > 125cc,Conductor,De 35 a 39 años,Hombre,"444950,58","4472305,63",N,0.0,0.0,3,0.0,1.0,0.0,0.0,-49.876258,-3.738781,33.895096,0.0,5.6,10.3,15.0,0.600000,4.400000,0,0,0,0,0,0,0,2,10,4,2019,1.345597,7.649846,60.629284,7707,3511,68.702086,31.297914,26.853430,26.904178,21776.201095,57.305792,31.033757,120.935777,2.508042,28.620910,1.970405e+06,55.048458,31.883662,126.350828,0.041220
108279,2020S010132,2020-07-20,Punta,AVDA. PORTUGAL,LATINA,Solo salida de

In [29]:
df9["hora_densidad_ponderado"]= df9["densidad_accidente"] / (df9["amax_y"] - df9["amin_y"]) * 100
df9.sort_values(by="hora_densidad_ponderado",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,sum_x,mean_x,std_x,amax_x,amin_x,tipo_accidente_densidad_ponderado,sum_y,mean_y,std_y,amax_y,amin_y,hora_densidad_ponderado
2622,2020S016218,2020-11-13,Noche,AVDA. CANILLEJAS A VICALVARO / CALL. ALBAIDA,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,Nublado,Furgoneta,Conductor,De 35 a 39 años,Hombre,"448130,88","4475641,03",N,1.0,0.0,14,1.0,0.0,0.0,0.0,-49.873508,-3.812494,126.350828,0.0,22.8,30.2,37.5,1.7,8.1,1,0,0,0,0,0,0,2,13,11,2020,0.629994,8.192649,60.629284,7707,3511,68.702086,31.297914,100.345741,100.332520,1.474194e+06,55.116244,31.791005,126.350828,0.055007,100.043554,7.104302e+05,54.783331,31.938207,126.350828,0.105714,100.083737
49802,2019S038212,2019-11-12,Valle,AUTOV. M-30,CIUDAD LINEAL,Colisión lateral,Lluvia débil,Motocicleta hasta 125cc,Conductor,De 18 a 20 años,Hombre,"444082,67","4475001,14",N,0.0,1.0,7,0.0,1.0,0.0,0.0,-49.873508,-3.812494,126.350828,0.0,18.4,24.3,30.2,1.7,4.7,0,0,0,0,0,0,0,2,12,11,2019,0.629994,8.192649,60.629284,2123,1182,64.216576,35.753176,100.081620,100.262927,8.337262e+05,54.861235,31.785113,126.350828,0.041224,100.032638,3.265187e+06,54.857724,31.963052,126.350828,0.082441,100.065290
25741,2020S016219,2020-11-13,Valle,AVDA. ARCENTALES / PLAZA. GRECIA,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,Despejado,Furgoneta,Conductor,De 30 a 34 años,Hombre,446822,"4475148,08",N,1.0,0.0,14,1.0,0.0,0.0,0.0,-49.873508,-3.812494,126.350828,0.0,22.8,30.2,37.5,1.7,8.1,1,0,0,0,0,0,0,3,13,11,2020,0.629994,8.192649,60.629284,7638,3740,67.111853,32.861787,100.345741,100.332520,1.474194e+06,55.116244,31.791005,126.350828,0.055007,100.043554,3.265187e+06,54.857724,31.963052,126.350828,0.082441,100.065290
56067,2020S013341,2020-09-27,Valle,CALL. ALCALDE SAINZ DE BARANDA,RETIRO,Alcance,NaN,Turismo,Pasajero,De 45 a 49 años,Mujer,"443563,28","4474487,91",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.873508,-3.812494,126.350828,0.0,23.1,30.1,37.1,1.9,9.2,1,0,0,0,0,0,0,3,27,9,2020,1.937179,7.047242,28.727172,7256,3145,69.762523,30.237477,100.032638,100.032638,1.376888e+06,54.657915,32.067828,126.350828,0.082441,100.065290,3.265187e+06,54.857724,31.963052,126.350828,0.082441,100.065290
61353,2020S013342,2020-09-27,Valle,AUTOV. M-30,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 50 a 54 años,Mujer,"443612,26","4478892,7",N,0.0,0.0,6.0,0.0,0.0,0.0,1.0,-49.873508,-3.812494,126.350828,0.0,23.1,30.1,37.1,1.9,9.2,1,0,0,0,0,0,0,2,27,9,2020,1.937179,7.047242,28.727172,6257,2570,70.876756,29.111917,100.154162,100.032638,1.376888e+06,54.657915,32.067828,126.350828,0.082441,100.065290,3.265187e+06,54.857724,31.963052,126.350828,0.082441,100.065290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60442,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,"436511,35","4478908,8",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.5,12.8,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,60.629284,5037,1928,72.318737,27.681263,0.065312,0.065269,1.376888e+06,54.657915,32.067828,126.350828,0.082441,0.065290,3.265187e+06,54.857724,31.963052,126.350828,0.082441,0.065290
85739,2019S025634,2019-04-07,Punta,CALL. DOCTOR ESQUERDO,RETIRO,Colisión múltiple,Despejado,Turismo

In [30]:
# eliminamos columnas que han surgido
df9.drop(['sum_x','mean_x','std_x','amax_x','amin_x','sum_y','mean_y','std_y','amax_y','amin_y'],axis=1,inplace=True)
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679
2,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,8.018739,7.650104
3,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Peatón,De 35 a 39 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,7707,3511,68.702086,31.297914,46.215468,46.119247,48.366287,46.142806
4,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108278,2019S032026,2019-04-10,Punta,AUTOV. A-3,PUENTE DE VALLECAS,Solo salida de la vía,Despejado,Motocicleta > 125cc,Conductor,De 35 a 39 años,Hombre,"444950,58","4472305,63",N,0.0,0.0,3,0.0,1.0,0.0,0.0,-49.876258,-3.738781,33.895096,0.0,5.6,10.3,15.0,0.600000,4.400000,0,0,0,0,0,0,0,2,10,4,2019,1.345597,7.649846,60.629284,7707,3511,68.702086,31.297914,26.853430,26.904178,28.620910,26.834931
108279,2020S010132,2020-07-20,Punta,AVDA. PORTUGAL,LATINA,Solo salida de la vía,Despejado,Ciclomotor,Conductor,De 21 a 24 años,Hombre,"437414,7","4473666,39",N,0.0,0.0,14.0,0.0,0.0,1.0,0.0,-49.918775,-3.772830,32.172243,0.0,21.6,28.8,36.1,2.014238,9.147069,1,0,0,0,0,0,0,2,20,7,2020,1.425777,9.017894,60.629284,4905,2400,67.145791,32.854209,25.580477,25.672072,27.166139,25.470939
108280,2020S010132,2020-07-20,Punta,AVDA. PORTUGAL,LATINA,Solo salida de la vía,Despejado,Ciclomotor,Pasajero,De 21 a 24 años,Hombre,"437414,7","4473666,39",N,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-49.870496,-3.788338,98.374280,0.0,21.6,28.8,36.1,2.014238,9.147069,1,0,0,0,0,0,0,2,20,7,2020,1.425777,9.017894,60.629284,4905,2400,67.145791,32.854209,78.2183

In [31]:
datosLC = df9.groupby("localizacion")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosLC

,localizacion,sum,mean,std,amax,amin
0,GTA. CUATRO CAMINOS / CALL. SANTA ENGRACIA,378.682424,126.227475,0.0,126.227475,126.227475
1,CALL. PUERTO VALLARTA,126.162530,126.162530,NaN,126.162530,126.162530
2,PALAC. CRISTAL,125.984021,125.984021,NaN,125.984021,125.984021
3,CMNO. PERALES / CALL. ANTONIO LOPEZ,250.963089,125.481544,0.0,125.481544,125.481544
4,CALL. VALDERROBRES / CALL. DISCOBOLO,250.926395,125.463198,0.0,125.463198,125.463198
...,...,...,...,...,...,...
14899,CALL. MESON DE PAREDES / PLAZA. NELSON MANDELA,1.081055,1.081055,NaN,1.081055,1.081055
14900,CALL. MARIANO SERRANO,0.840473,0.840473,NaN,0.840473,0.840473
14901,CALL. AZUCENAS / CALL. BRAVO MURILLO,0.840473,0.840473,NaN,0.840473,0.840473
14902,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,0.995816,0.497908,0.0,0.497908,0.497908


In [32]:
df9=pd.merge(df9, datosLC, on='localizacion')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,sum,mean,std,amax,amin
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,27.637414,27.637414,NaN,27.637414,27.637414
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.9,6.1,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,63.309126,31.654563,31.108003,53.651243,9.657883
2,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.9,6.1,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,8.018739,7.650104,63.309126,31.654563,31.108003,53.651243,9.657883
3,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Peatón,De 35 a 39 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.8,16.7,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,7707,3511,68.702086,31.297914,46.215468,46.119247,48.366287,46.142806,120.446421,30.111605,32.494928,58.253038,1.970172
4,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.8,16.7,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,120.446421,30.111605,32.494928,58.253038,1.970172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108243,2020S013127,2020-09-24,Punta,CALL. MANCHESTER / CALL. ESTOCOLMO,SAN BLAS-CANILLEJAS,Solo salida de la vía,Lluvia débil,Turismo,Conductor,De 21 a 24 años,Hombre,"449035,02","4475662,27",S,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.807087,-3.697043,3.897333,0.0,10.1,14.4,18.7,3.3,14.4,1,0,0,0,0,0,0,3,24,9,2020,1.937179,7.047242,60.629284,4905,2400,67.145791,32.854209,3.095197,3.085539,3.290895,3.085539,88.609814,29.536605,44.408522,80.815149,3.897333
108244,2020S013127,2020-09-24,Punta,CALL. MANCHESTER / CALL. ESTOCOLMO,SAN BLAS-CANILLEJAS,Solo salida de la vía,Lluvia débil,Turismo,Pasajero,De 21 a 24 años,Hombre,"449035,02","4475662,27",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.807087,-3.697043,3.897333,0.0,10.1,14.4,18.7,3.3,14.4,1,0,0,0,0,0,0,3,24,9,2020,1.937179,7.047242,60.629284,4905,2400,67.145791,32.854209,3.095197,3.085539,3.290895,3.085539,88.609814,29.536605,44.408522,80.815149,3

In [33]:
df9["localizacion_densidad_pondracion"]= df9["densidad_accidente"] / (df9["amax"] - df9["amin"]) * 100
df9.sort_values(by="localizacion_densidad_pondracion",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,sum,mean,std,amax,amin,localizacion_densidad_pondracion
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,27.637414,27.637414,NaN,27.637414,27.637414,inf
85746,2019S004603,2019-01-15,Valle,CALL. NOVICIADO / CALL. SAN BERNARDO,CENTRO,Choque contra obstáculo fijo,Despejado,Camión rígido,Conductor,De 35 a 39 años,Hombre,"439979,25","4475240,65",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.823159,-3.973688,2.508042,0.0,25.0,31.8,38.5,1.7,7.8,0,0,0,0,0,0,0,2,15,1,2019,1.184984,9.158655,60.629284,7707,3511,68.702086,31.297914,1.990796,2.009225,1.987282,1.986279,5.016085,2.508042,0.000000,2.508042,2.508042,inf
85700,2020S006099,2020-04-15,Valle,CALL. NOVENA / CALL. CUARTA,CHAMARTÍN,Choque contra obstáculo fijo,Lluvia débil,Camión rígido,Conductor,De 60 a 64 años,Hombre,"443256,54","4478518,55",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.824918,-3.814519,50.020034,0.0,25.0,31.8,38.5,1.7,7.8,1,0,0,0,0,0,0,1,15,4,2020,1.345597,7.649846,60.629284,3072,1246,71.144048,28.855952,39.649243,40.071700,39.634072,39.614060,50.020034,50.020034,NaN,50.020034,50.020034,inf
85706,2019S034461,2019-10-25,Valle,CALL. ATOCHA / CALL. SAN EUGENIO,CENTRO,Choque contra obstáculo fijo,Despejado,Camión rígido,Conductor,De 25 a 29 años,Hombre,"440825,85","4473650,42",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.826888,-3.850541,27.158347,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,25,10,2019,1.182292,7.873407,60.629284,7702,4034,65.621539,34.369941,21.557345,21.756905,21.519295,21.508429,27.158347,27.158347,NaN,27.158347,27.158347,inf
85707,2021S003662,2021-11-03,Valle,CALL. SANTA ANA / CALL. RODRIGO DE GUEVARA,CENTRO,Choque contra obstáculo fijo,Despejado,Camión rígido,Conductor,De 40 a 44 años,Hombre,"439854,43","4473447,46",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.863568,-3.813310,117.052026,0.0,17.9,24.8,31.6,1.7,6.7,1,0,0,0,0,0,0,1,3,11,2021,0.629994,8.192649,60.629284,8283,3400,70.897886,29.102114,92.911801,93.771900,92.747806,92.700975,117.052026,117.052026,NaN,117.052026,117.052026,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45755,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,"436511,35","4478908,8",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.5,12.8,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,60.629284,5037,1928,72.318737,27.681263,0.065312,0.065269,0.065290,0.065290,3106.921040,50.933132,32.363382,118.599164,0.082441,0.069560
35017,2019S025634,2019-04-07,Punta,CALL. DOCTOR ESQUERDO,RETIRO,Colisión múltiple,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,"443257,61","4474572,47",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.919575,-3.964384,0.060811,4.5,16.0,22.0,28.0,2.8,11.7,0,0,0,0,0,0,0,4,7,4,2019,1.345597,7.649846,28.727172,7707,3511,68.702086,31.297914,0.048145,0.048145,0.048185,0.048145,17210.992457,54.987196,32.877383,123.512387,0.060

In [34]:
datosTP = df9.groupby("tipo_persona")['densidad_accidente'].agg([np.sum, np.mean, np.std, np.max,np.min]).sort_values(by="mean",ascending=False).reset_index()
datosTP

,tipo_persona,sum,mean,std,amax,amin
0,Conductor,4.816843e+06,54.949151,31.938102,126.350828,0.041220
1,Pasajero,9.290044e+05,54.876509,31.913462,126.350828,0.060811
2,Peatón,1.980602e+05,54.277947,31.910694,126.208812,0.189648


In [35]:
df9=pd.merge(df9, datosTP, on='tipo_persona')
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,sum_x,mean_x,std_x,amax_x,amin_x,localizacion_densidad_pondracion,sum_y,mean_y,std_y,amax_y,amin_y
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,27.637414,27.637414,NaN,27.637414,27.637414,inf,4.816843e+06,54.949151,31.938102,126.350828,0.041220
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.9,6.1,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,63.309126,31.654563,31.108003,53.651243,9.657883,121.953046,4.816843e+06,54.949151,31.938102,126.350828,0.041220
2,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.8,16.7,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,120.446421,30.111605,32.494928,58.253038,1.970172,103.500483,4.816843e+06,54.949151,31.938102,126.350828,0.041220
3,2019S031541,2019-09-30,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 30 a 34 años,Hombre,"436115,02","4469290,03",N,1.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.817487,-3.712392,1.970172,5.4,24.0,29.5,35.0,2.8,10.3,0,0,0,0,0,0,0,2,30,9,2019,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.559796,1.561094,1.560593,120.446421,30.111605,32.494928,58.253038,1.970172,3.500483,4.816843e+06,54.949151,31.938102,126.350828,0.041220
4,2021S006936,2021-10-04,Noche,CALL. SINFONIA,LATINA,Atropello a animal,NaN,Turismo,Conductor,De 35 a 39 años,Hombre,"434421,47","4469447,63",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.863781,-3.847351,69.122402,0.0,9.0,15.5,22.0,1.9,13.6,0,0,0,0,0,0,0,1,4,10,2021,1.182292,7.873407,60.629284,7707,3511,68.702086,31.297914,54.959924,54.724582,57.390895,54.752537,900.526894,69.271300,19.899171,85.875412,26.692837,116.795191,4.816843e+06,54.949151,31.938102,126.350828,0.041220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,2021S007301,2021-04-30,Punta,CALL. ALICANTE,ARGANZUELA,Caída,Despejado,Motocicleta > 125cc,Peatón,De 10 a 14 años,Hombre,"441212,68","4471754,45",N,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.849445,-3.819971,95.360643,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,4,30,4,2021,1.345597,7.649846,60.629284,549,505,52.087287,47.912713,75.607533,75.692358,75.523340,75.497538,317.030292,79.257573,18.594224,95.360643,63.154503,296.094600,1.980602e+05,54.277947,31.910694,126.208812,0.189648
108234,2021S007301,2021-04-30,Punta,

In [36]:
df9["tipo_persona_densidad_ponderado"]= df9["densidad_accidente"] / (df9["amax_y"] - df9["amin_y"]) * 100
df9.sort_values(by="tipo_persona_densidad_ponderado",ascending=False)

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,sum_x,mean_x,std_x,amax_x,amin_x,localizacion_densidad_pondracion,sum_y,mean_y,std_y,amax_y,amin_y,tipo_persona_densidad_ponderado
107017,2020S007504,2020-05-30,Valle,CALL. CARTAGENA / CALL. FRANCISCO SILVELA,SALAMANCA,Atropello a persona,Despejado,Furgoneta,Peatón,De 65 a 69 años,Mujer,"443133,31","4475770,28",N,1.0,0.0,3,1.0,0.0,0.0,0.0,-49.873507,-3.811977,126.208812,4.2,21.6,27.8,34.0,2.8,9.7,0,0,0,0,0,0,0,2,30,5,2020,0.868112,9.293896,28.727172,1383,755,64.686623,35.313377,99.971009,100.219749,100.207267,99.952819,377.577569,62.929595,50.967767,126.208812,16.033006,114.552202,1.980602e+05,54.277947,31.910694,126.208812,0.189648,100.150491
105906,2019S036329,2019-11-21,Punta,CALL. MAR AMARILLO,HORTALEZA,Atropello a persona,Lluvia débil,Turismo,Peatón,De 45 a 49 años,Mujer,"445643,86","4480635,93",N,0.0,0.0,7,0.0,0.0,0.0,1.0,-49.872640,-3.811549,126.162530,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,21,11,2019,0.629994,8.192649,28.727172,7256,3145,69.762523,30.237477,100.010621,99.883562,100.170520,99.883558,618.027358,77.253420,44.985764,126.162530,21.245962,120.250340,1.980602e+05,54.277947,31.910694,126.208812,0.189648,100.113766
100845,2020S016033,2020-11-11,Punta,PLAZA. CORREGIDOR SANCHO DE CORDOBA,MORATALAZ,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,Pasajero,De 50 a 54 años,Mujer,"444592,65","4473244,54",N,0.0,0.0,7,0.0,1.0,0.0,0.0,-49.873508,-3.812494,126.350828,0.0,18.4,24.3,30.2,1.7,4.7,1,0,0,0,0,0,0,3,11,11,2020,0.629994,8.192649,28.727172,6257,2570,70.876756,29.111917,100.222976,100.262927,100.043554,100.032634,870.918139,72.576512,30.404062,126.350828,28.510403,129.139696,9.290044e+05,54.876509,31.913462,126.350828,0.060811,100.048152
94341,2020S013341,2020-09-27,Valle,CALL. ALCALDE SAINZ DE BARANDA,RETIRO,Alcance,NaN,Turismo,Pasajero,De 45 a 49 años,Mujer,"443563,28","4474487,91",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.873508,-3.812494,126.350828,0.0,23.1,30.1,37.1,1.9,9.2,1,0,0,0,0,0,0,3,27,9,2020,1.937179,7.047242,28.727172,7256,3145,69.762523,30.237477,100.032638,100.032638,100.065290,100.065290,3766.643746,60.752318,36.172454,126.350828,4.193811,103.433132,9.290044e+05,54.876509,31.913462,126.350828,0.060811,100.048152
91447,2020S013336,2020-09-27,Valle,AVDA. ENTREVIAS,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 18 a 20 años,Mujer,"443652,27","4469911,44",N,0.0,0.0,7.0,0.0,0.0,0.0,1.0,-49.873508,-3.812494,126.350828,0.0,23.1,30.1,37.1,1.9,9.2,1,0,0,0,0,0,0,3,27,9,2020,1.937179,7.047242,28.727172,2123,1182,64.216576,35.753176,100.101593,100.032638,100.043554,100.065290,6272.052622,54.069419,34.475414,126.350828,0.902783,100.719647,9.290044e+05,54.876509,31.913462,126.350828,0.060811,100.048152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36880,2019S029484,2019-04-09,Valle,AVDA. PADRE HUIDOBRO,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 55 a 59 años,Mujer,"436511,35","4478908,8",N,0.0,0.0,1,0.0,0.0,0.0,1.0,-49.724267,-3.832156,0.082441,0.0,22.0,28.6,35.2,2.5,12.8,1,0,0,0,0,0,0,2,9,4,2019,1.345597,7.649846,28.727172,5037,1928,72.318737,27.681263,0.065312,0.065269,0.065290,0.065290,3106.921040,50.933132,32.36338

In [37]:
# eliminamos columnas que han surgido
df9.drop(['sum_x','mean_x','std_x','amax_x','amin_x','sum_y','mean_y','std_y','amax_y','amin_y'],axis=1,inplace=True)
df9

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,localizacion_densidad_pondracion,tipo_persona_densidad_ponderado
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.3,5.3,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,inf,21.880690
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.9,6.1,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,121.953046,42.475980
2,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.8,16.7,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,103.500483,46.119246
3,2019S031541,2019-09-30,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 30 a 34 años,Hombre,"436115,02","4469290,03",N,1.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.817487,-3.712392,1.970172,5.4,24.0,29.5,35.0,2.8,10.3,0,0,0,0,0,0,0,2,30,9,2019,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.559796,1.561094,1.560593,3.500483,1.559796
4,2021S006936,2021-10-04,Noche,CALL. SINFONIA,LATINA,Atropello a animal,NaN,Turismo,Conductor,De 35 a 39 años,Hombre,"434421,47","4469447,63",N,0.0,0.0,14,0.0,0.0,0.0,1.0,-49.863781,-3.847351,69.122402,0.0,9.0,15.5,22.0,1.9,13.6,0,0,0,0,0,0,0,1,4,10,2021,1.182292,7.873407,60.629284,7707,3511,68.702086,31.297914,54.959924,54.724582,57.390895,54.752537,116.795191,54.724580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,2021S007301,2021-04-30,Punta,CALL. ALICANTE,ARGANZUELA,Caída,Despejado,Motocicleta > 125cc,Peatón,De 10 a 14 años,Hombre,"441212,68","4471754,45",N,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.849445,-3.819971,95.360643,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,4,30,4,2021,1.345597,7.649846,60.629284,549,505,52.087287,47.912713,75.607533,75.692358,75.523340,75.497538,296.094600,75.671541
108234,2021S007301,2021-04-30,Punta,CALL. ALICANTE,ARGANZUELA,Caída,Despejado,Motocicleta > 125cc,Peatón,De 10 a 14 años,Hombre,"441212,68","4471754,45",N,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.895391,-3.802393,63.154503,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,4,30,4,2021,1.345597,7.649846,60.629284,549,505,52.087287,47.912713,50.072608,50.128785,50.016850,49.999762,196.094600,50.114999
108235,2021S007301,2021-04-30,Punta,CALL. ALICANTE,ARGANZUELA,Caída,Despejado,Motocicleta > 125cc,Peatón,De 10 a 14 años,Hombre,"441212,68","4471754,45",N,0.0,0.0,14,0.0,1.0,0.0,0.0,-49.895391,-3.802393,63.154503,0.0,6.3,13.9,21.5,0.6,7.2,1,0,0,0,0,0,0,4,30,4,2021,1.345597,7.6498

In [38]:
distrito_accidentes=df9.groupby(df9['distrito'])['num_expediente'].count().reset_index()
distrito_accidentes['%accidentes_distrito']=(distrito_accidentes['num_expediente']/(distrito_accidentes['num_expediente'].sum()))*100
distrito_accidentes=distrito_accidentes[['distrito','%accidentes_distrito']]
df10=pd.merge(df9,distrito_accidentes,on='distrito')
df10

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,localizacion_densidad_pondracion,tipo_persona_densidad_ponderado,%accidentes_distrito
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,inf,21.880690,6.617824
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,121.953046,42.475980,6.617824
2,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,103.500483,46.119246,6.617824
3,2019S031541,2019-09-30,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 30 a 34 años,Hombre,"436115,02","4469290,03",N,1.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.817487,-3.712392,1.970172,5.4,24.0,29.5,35.0,2.800000,10.300000,0,0,0,0,0,0,0,2,30,9,2019,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.559796,1.561094,1.560593,3.500483,1.559796,6.617824
4,2021S009536,2021-01-16,Noche,CALL. GENERAL RICARDOS / CALL. CLARA CAMPOAMOR,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"438470,62","4471582,84",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.865929,-3.779119,93.482847,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,16,1,2021,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,74.165290,74.010878,74.018955,74.048685,167.315735,74.010876,6.617824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,2019S036088,2019-11-19,Punta,AVDA. LOS ROSALES,VILLAVERDE,Atropello a persona,Despejado,Turismo,Peatón,De 65 a 69 años,Hombre,"442438,96","4467021,44",N,0.0,1.0,5,0.0,0.0,0.0,1.0,-49.903697,-3.837109,66.262706,0.0,8.5,11.5,14.5,3.900000,13.300000,1,0,0,0,0,0,0,2,19,11,2019,0.629994,8.192649,60.629284,1383,755,64.686623,35.313377,52.578773,52.460545,52.611260,52.460543,201.897460,52.581452,2.997099
108234,2020S010026,2020-07-18,Punta,AVDA. GRAN VIA DE VILLAVERDE / AVDA. GRAN VIA DE VILLAVERDE,VILLAVERDE,Atropello a persona,Despejado,Turismo,Peatón,De 60 a 64 años,Mujer,"440244,71","4465862,51",N,1.0,0.0,3,0.0,0.0,0.0,1.0,-49.860982,-3.752279,52.767925,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,2,18,7,2020,1.425777,9.017894,28.727172,3072,1246,71.144048,28.855952,41.870803,41.776653,41.896675,41.776652,inf,41.872937,2.997099
108235,2021S003865,2021-03-13,Punta,CALL. DOMING

In [39]:
localizacion_accidentes=df10.groupby(df10['localizacion'])['num_expediente'].count().reset_index()
localizacion_accidentes['%localizacion_distrito']=(localizacion_accidentes['num_expediente']/(localizacion_accidentes['num_expediente'].sum()))*100
localizacion_accidentes=localizacion_accidentes[['localizacion','%localizacion_distrito']]
df11=pd.merge(df10,localizacion_accidentes,on='localizacion')
df11

,num_expediente,fecha2,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,localizacion_densidad_pondracion,tipo_persona_densidad_ponderado,%accidentes_distrito,%localizacion_distrito
0,2019S003289,2019-01-01,Noche,AVDA. POBLADOS / CMNO. CRUCES,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"436012,37","4469875,33",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,inf,21.880690,6.617824,0.000924
1,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,121.953046,42.475980,6.617824,0.001848
2,2020S017728,2020-04-12,Noche,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,CARABANCHEL,Atropello a animal,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,"437942,75","4472333,78",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,8.018739,7.650104,21.953046,7.647384,6.617824,0.001848
3,2020S006260,2020-04-23,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Atropello a animal,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,"436122,05","4469300,08",N,1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,103.500483,46.119246,6.617824,0.003696
4,2019S031541,2019-09-30,Noche,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,CARABANCHEL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 30 a 34 años,Hombre,"436115,02","4469290,03",N,1.0,0.0,14.0,0.0,0.0,0.0,1.0,-49.817487,-3.712392,1.970172,5.4,24.0,29.5,35.0,2.800000,10.300000,0,0,0,0,0,0,0,2,30,9,2019,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.559796,1.561094,1.560593,3.500483,1.559796,6.617824,0.003696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,2019S035049,2019-07-11,Punta,AVDA. OROVILLA / CALL. GENEROSIDAD,VILLAVERDE,Caída,Despejado,Autobús,Conductor,De 30 a 34 años,Hombre,"441410,6","4468142,24",N,1.0,0.0,14,0.0,0.0,1.0,0.0,-49.858100,-3.805270,102.104324,0.0,18.5,25.3,32.1,2.014238,9.147069,1,0,0,0,0,0,0,3,11,7,2019,1.425777,9.017894,60.629284,7638,3740,67.111853,32.861787,81.018726,81.021119,80.864173,80.836546,912.484774,80.836546,2.997099,0.002772
108234,2019S035049,2019-07-11,Punta,AVDA. OROVILLA / CALL. GENEROSIDAD,VILLAVERDE,Caída,Despejado,Autobús,Pasajero,De 35 a 39 años,Mujer,"441410,6","4468142,24",N,1.0,0.0,6,0.0,0.0,1.0,0.0,-49.866270,-3.839917,90.914622,0.0,18.5,25.3,32.1,2.014238,9.147069,1,0,0,0,0,0,0,3,11,7,2019,1.425777,9.017894,28.727172,7707,3511,68.702086,31.297914,72.139814,72.141944,72.002198,71.977599

In [40]:
# Realizamos los get_dummies
df12=pd.get_dummies(df11,columns=['hora','tipo_accidente','rango_edad','tipo_persona','sexo',"positiva_alcohol",'distrito'])
df12

,num_expediente,fecha2,localizacion,estado_meteorológico,tipo_vehiculo,coordenada_x_utm,coordenada_y_utm,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,lat,lon,densidad_accidente,prec,tmin,tmed,tmax,velmedia,racha,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,dia,mes,año,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_vehiculo_densidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,localizacion_densidad_pondracion,tipo_persona_densidad_ponderado,%accidentes_distrito,%localizacion_distrito,hora_Noche,hora_Punta,hora_Valle,tipo_accidente_Alcance,tipo_accidente_Atropello a animal,tipo_accidente_Atropello a persona,tipo_accidente_Caída,tipo_accidente_Choque contra obstáculo fijo,tipo_accidente_Colisión frontal,tipo_accidente_Colisión fronto-lateral,tipo_accidente_Colisión lateral,tipo_accidente_Colisión múltiple,tipo_accidente_Despeñamiento,tipo_accidente_Otro,tipo_accidente_Solo salida de la vía,tipo_accidente_Vuelco,rango_edad_De 10 a 14 años,rango_edad_De 15 a 17 años,rango_edad_De 18 a 20 años,rango_edad_De 21 a 24 años,rango_edad_De 25 a 29 años,rango_edad_De 30 a 34 años,rango_edad_De 35 a 39 años,rango_edad_De 40 a 44 años,rango_edad_De 45 a 49 años,rango_edad_De 50 a 54 años,rango_edad_De 55 a 59 años,rango_edad_De 6 a 9 años,rango_edad_De 60 a 64 años,rango_edad_De 65 a 69 años,rango_edad_De 70 a 74 años,rango_edad_Desconocido,rango_edad_Menor de 5 años,rango_edad_Más de 74 años,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón,sexo_Desconocido,sexo_Hombre,sexo_Mujer,positiva_alcohol_N,positiva_alcohol_S,distrito_ARGANZUELA,distrito_BARAJAS,distrito_CARABANCHEL,distrito_CENTRO,distrito_CHAMARTÍN,distrito_CHAMBERÍ,distrito_CIUDAD LINEAL,distrito_FUENCARRAL-EL PARDO,distrito_HORTALEZA,distrito_LATINA,distrito_MONCLOA-ARAVACA,distrito_MORATALAZ,distrito_PUENTE DE VALLECAS,distrito_RETIRO,distrito_SALAMANCA,distrito_SAN BLAS-CANILLEJAS,distrito_TETUÁN,distrito_USERA,distrito_VICÁLVARO,distrito_VILLA DE VALLECAS,distrito_VILLAVERDE
0,2019S003289,2019-01-01,AVDA. POBLADOS / CMNO. CRUCES,Despejado,Turismo,"436012,37","4469875,33",1.0,0.0,14,0.0,0.0,0.0,1.0,-49.920070,-3.891375,27.637414,0.0,1.4,6.6,11.7,0.300000,5.300000,1,0,0,0,0,0,0,1,1,1,2019,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,21.880691,22.946770,21.891868,inf,21.880690,6.617824,0.000924,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020S017728,2020-04-12,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,Despejado,Turismo,"437942,75","4472333,78",1.0,0.0,14,0.0,0.0,0.0,1.0,-49.904813,-3.820553,53.651243,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,42.475981,44.545513,42.497679,121.953046,42.475980,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020S017728,2020-04-12,PASEO. ERMITA DEL SANTO / Gasolinera BP Fernán Fernández,Despejado,Turismo,"437942,75","4472333,78",1.0,0.0,14,0.0,0.0,0.0,1.0,-49.829297,-3.673215,9.657883,0.0,17.2,24.0,30.8,1.900000,6.100000,1,0,0,0,0,0,0,2,12,4,2020,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,7.646198,8.018739,7.650104,21.953046,7.647384,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020S006260,2020-04-23,AVDA. CARABANCHEL ALTO / TRVA. GUITARRA,Despejado,Turismo,"436122,05","4469300,08",1.0,0.0,14,0.0,0.0,0.0,1.0,-49.867112,-3.754383,58.253038,0.1,11.0,20.5,30.0,2.800000,16.700000,1,0,0,0,0,0,0,2,23,4,2020,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,46.119247,48.366287,46.142806,103.500483

In [41]:
df12.drop(['dia','mes','año','coordenada_x_utm','coordenada_y_utm','num_expediente','fecha2','estado_meteorológico','tipo_vehiculo','localizacion','tmax','tmin','racha','velmedia','lat','lon','tipo_vehiculo_densidad_ponderado','localizacion_densidad_pondracion'],axis=1,inplace=True)
df12

,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,densidad_accidente,prec,tmed,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,tipo_persona_densidad_ponderado,%accidentes_distrito,%localizacion_distrito,hora_Noche,hora_Punta,hora_Valle,tipo_accidente_Alcance,tipo_accidente_Atropello a animal,tipo_accidente_Atropello a persona,tipo_accidente_Caída,tipo_accidente_Choque contra obstáculo fijo,tipo_accidente_Colisión frontal,tipo_accidente_Colisión fronto-lateral,tipo_accidente_Colisión lateral,tipo_accidente_Colisión múltiple,tipo_accidente_Despeñamiento,tipo_accidente_Otro,tipo_accidente_Solo salida de la vía,tipo_accidente_Vuelco,rango_edad_De 10 a 14 años,rango_edad_De 15 a 17 años,rango_edad_De 18 a 20 años,rango_edad_De 21 a 24 años,rango_edad_De 25 a 29 años,rango_edad_De 30 a 34 años,rango_edad_De 35 a 39 años,rango_edad_De 40 a 44 años,rango_edad_De 45 a 49 años,rango_edad_De 50 a 54 años,rango_edad_De 55 a 59 años,rango_edad_De 6 a 9 años,rango_edad_De 60 a 64 años,rango_edad_De 65 a 69 años,rango_edad_De 70 a 74 años,rango_edad_Desconocido,rango_edad_Menor de 5 años,rango_edad_Más de 74 años,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón,sexo_Desconocido,sexo_Hombre,sexo_Mujer,positiva_alcohol_N,positiva_alcohol_S,distrito_ARGANZUELA,distrito_BARAJAS,distrito_CARABANCHEL,distrito_CENTRO,distrito_CHAMARTÍN,distrito_CHAMBERÍ,distrito_CIUDAD LINEAL,distrito_FUENCARRAL-EL PARDO,distrito_HORTALEZA,distrito_LATINA,distrito_MONCLOA-ARAVACA,distrito_MORATALAZ,distrito_PUENTE DE VALLECAS,distrito_RETIRO,distrito_SALAMANCA,distrito_SAN BLAS-CANILLEJAS,distrito_TETUÁN,distrito_USERA,distrito_VICÁLVARO,distrito_VILLA DE VALLECAS,distrito_VILLAVERDE
0,1.0,0.0,14,0.0,0.0,0.0,1.0,27.637414,0.0,6.6,1,0,0,0,0,0,0,1,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,22.946770,21.891868,21.880690,6.617824,0.000924,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,14,0.0,0.0,0.0,1.0,53.651243,0.0,24.0,1,0,0,0,0,0,0,2,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,44.545513,42.497679,42.475980,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,14,0.0,0.0,0.0,1.0,9.657883,0.0,24.0,1,0,0,0,0,0,0,2,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,8.018739,7.650104,7.647384,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,14,0.0,0.0,0.0,1.0,58.253038,0.1,20.5,1,0,0,0,0,0,0,2,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,48.366287,46.142806,46.119246,6.617824,0.003696,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,14.0,0.0,0.0,0.0,1.0,1.970172,5.4,29.5,0,0,0,0,0,0,0,2,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.561094,1.560593,1.559796,6.617824,0.003696,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,1.0,0.0,14,0.0,0.0,1.0,0.0,102.104324,0.0,25.3,1,0,0,0,0,0,0,3,1.425777,9.017894,60.629284,7638,3740,67.111853,32.861787,81.0187

In [42]:
df12=df12.fillna(0)
df12

,Interseccion,AltaVelocidad,lesividad,vehiculo_furgoneta,vehiculo_motocilceta,vehiculo_resto,vehiculo_turismo,densidad_accidente,prec,tmed,vrf_Despejado,vrf_Lluvioso,vrf_Granizando,vrf_Nevando,vrf_Lluvia débil,vrf_Se_desconoce,vrf_Nublado,n_victimas,media_mes_prec,%accidente_mes,%sexo,Hombre,Mujer,%hombre,%Mujer,distrito_desnsidad_ponderado,tipo_accidente_densidad_ponderado,hora_densidad_ponderado,tipo_persona_densidad_ponderado,%accidentes_distrito,%localizacion_distrito,hora_Noche,hora_Punta,hora_Valle,tipo_accidente_Alcance,tipo_accidente_Atropello a animal,tipo_accidente_Atropello a persona,tipo_accidente_Caída,tipo_accidente_Choque contra obstáculo fijo,tipo_accidente_Colisión frontal,tipo_accidente_Colisión fronto-lateral,tipo_accidente_Colisión lateral,tipo_accidente_Colisión múltiple,tipo_accidente_Despeñamiento,tipo_accidente_Otro,tipo_accidente_Solo salida de la vía,tipo_accidente_Vuelco,rango_edad_De 10 a 14 años,rango_edad_De 15 a 17 años,rango_edad_De 18 a 20 años,rango_edad_De 21 a 24 años,rango_edad_De 25 a 29 años,rango_edad_De 30 a 34 años,rango_edad_De 35 a 39 años,rango_edad_De 40 a 44 años,rango_edad_De 45 a 49 años,rango_edad_De 50 a 54 años,rango_edad_De 55 a 59 años,rango_edad_De 6 a 9 años,rango_edad_De 60 a 64 años,rango_edad_De 65 a 69 años,rango_edad_De 70 a 74 años,rango_edad_Desconocido,rango_edad_Menor de 5 años,rango_edad_Más de 74 años,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón,sexo_Desconocido,sexo_Hombre,sexo_Mujer,positiva_alcohol_N,positiva_alcohol_S,distrito_ARGANZUELA,distrito_BARAJAS,distrito_CARABANCHEL,distrito_CENTRO,distrito_CHAMARTÍN,distrito_CHAMBERÍ,distrito_CIUDAD LINEAL,distrito_FUENCARRAL-EL PARDO,distrito_HORTALEZA,distrito_LATINA,distrito_MONCLOA-ARAVACA,distrito_MORATALAZ,distrito_PUENTE DE VALLECAS,distrito_RETIRO,distrito_SALAMANCA,distrito_SAN BLAS-CANILLEJAS,distrito_TETUÁN,distrito_USERA,distrito_VICÁLVARO,distrito_VILLA DE VALLECAS,distrito_VILLAVERDE
0,1.0,0.0,14,0.0,0.0,0.0,1.0,27.637414,0.0,6.6,1,0,0,0,0,0,0,1,1.184984,9.158655,60.629284,7702,4034,65.621539,34.369941,21.926341,22.946770,21.891868,21.880690,6.617824,0.000924,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,14,0.0,0.0,0.0,1.0,53.651243,0.0,24.0,1,0,0,0,0,0,0,2,1.345597,7.649846,60.629284,7702,4034,65.621539,34.369941,42.564601,44.545513,42.497679,42.475980,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,14,0.0,0.0,0.0,1.0,9.657883,0.0,24.0,1,0,0,0,0,0,0,2,1.345597,7.649846,28.727172,7702,4034,65.621539,34.369941,7.662151,8.018739,7.650104,7.647384,6.617824,0.001848,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,14,0.0,0.0,0.0,1.0,58.253038,0.1,20.5,1,0,0,0,0,0,0,2,1.345597,7.649846,60.629284,6257,2570,70.876756,29.111917,46.215468,48.366287,46.142806,46.119246,6.617824,0.003696,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,14.0,0.0,0.0,0.0,1.0,1.970172,5.4,29.5,0,0,0,0,0,0,0,2,1.937179,7.047242,60.629284,7638,3740,67.111853,32.861787,1.563050,1.561094,1.560593,1.559796,6.617824,0.003696,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108233,1.0,0.0,14,0.0,0.0,1.0,0.0,102.104324,0.0,25.3,1,0,0,0,0,0,0,3,1.425777,9.017894,60.629284,7638,3740,67.111853,32.861787,81.0187

In [43]:
df12.to_csv(("ACCIDENTALIDAD_TODO_NUMERO_090422.csv"), index=False)